Visual Dashboard:

In [41]:
# The following will be completed:
# Mark all launch sites on a map
# Mark the success/failed launches for each site on the map
# Calculate the distances between a launch site to its proximities
import folium
import wget
import pandas as pd
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

In [50]:
# For this dashboard, we need to know longitude and latitude,
# so I will need the original dataset titled: 'f9_data'
df=pd.read_csv('f9_data.csv')
df.head()

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,6,2010-06-04,Falcon 9,6123.547647,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857
1,8,2012-05-22,Falcon 9,525.000000,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857
2,10,2013-03-01,Falcon 9,677.000000,ISS,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857
3,11,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093
4,12,2013-12-03,Falcon 9,3170.000000,GTO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857


In [55]:
# I will apply the same code as before to add the 'landing_class' column
landing_outcomes = df['Outcome'].value_counts()
landing_outcomes

for i,outcome in enumerate(landing_outcomes.keys()):
    print(i,outcome)

bad_outcomes=set(landing_outcomes.keys()[[1,3,5,6,7]])

df['landing_class'] = df['Outcome'].apply(lambda x: 0 if x in bad_outcomes else 1)
df['landing_class'].value_counts()


0 True ASDS
1 None None
2 True RTLS
3 False ASDS
4 True Ocean
5 False Ocean
6 None ASDS
7 False RTLS


landing_class
1    60
0    30
Name: count, dtype: int64

In [56]:
# I will create a dataframe that includes lat and long at each launch site
# Select relevant sub-columns: `Launch Site`, `Latitude`, `Longitude`, `class`
spacex_df = df[['LaunchSite', 'Latitude', 'Longitude', 'landing_class']]
launch_sites_df = spacex_df.groupby(['LaunchSite'], as_index=False).first()
launch_sites_df = launch_sites_df[['LaunchSite', 'Latitude', 'Longitude']]
launch_sites_df

,LaunchSite,Latitude,Longitude
0,CCSFS SLC 40,28.561857,-80.577366
1,KSC LC 39A,28.608058,-80.603956
2,VAFB SLC 4E,34.632093,-120.610829


In [57]:
# Longitude and Latitude will be better displayed on a map, so I will use a folium map to do so
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [68]:
# I will create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# I will create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Add an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [59]:
# Now I will add a circle for each launch site in the data frame launch_sites_df
# Loop through each launch site and add a circle and marker

for index, row in launch_sites_df.iterrows():
    coordinate = [row['Latitude'], row['Longitude']]  # Extract Lat and Lon
    
    # Add a Circle
    circle = folium.Circle(
        location=coordinate,
        radius=1000,  # Radius in meters
        color='#d35400',
        fill=True
    ).add_child(folium.Popup(row['LaunchSite']))
    
    # Add a Marker with a text label
    marker = folium.map.Marker(
        location=coordinate,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % row['LaunchSite'],
        )
    )
    
    # Add the Circle and Marker to the map
    site_map.add_child(circle)
    site_map.add_child(marker)

# Display the map
site_map

In [69]:
# Now I will create markers for all launch records. If a launch was successful (class=1), 
# then we use a green marker and if a launch was failed, we use a red marker (class=0)

# Function to assign marker color based on landing class
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'

# Apply the function to the 'landing_class' column to create the 'marker_color' column
df['marker_color'] = df['landing_class'].apply(assign_marker_color)

# Display the last 10 rows to verify the result
df.tail(10)



,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,landing_class,marker_color
80,96,2020-06-13,Falcon 9,15600.000000,VLEO,CCSFS SLC 40,True ASDS,3,True,True,True,5e9e3032383ecb6bb234e7ca,5.0,5,B1059,-80.577366,28.561857,1,green
81,97,2020-06-30,Falcon 9,3880.000000,MEO,CCSFS SLC 40,True ASDS,1,True,False,True,5e9e3033383ecbb9e534e7cc,5.0,12,B1060,-80.577366,28.561857,1,green
82,98,2020-07-20,Falcon 9,6123.547647,GEO,CCSFS SLC 40,True ASDS,2,True,True,True,5e9e3033383ecbb9e534e7cc,5.0,13,B1058,-80.577366,28.561857,1,green
83,100,2020-08-18,Falcon 9,15600.000000,VLEO,CCSFS SLC 40,True ASDS,6,True,True,True,5e9e3032383ecb6bb234e7ca,5.0,9,B1049,-80.577366,28.561857,1,green
84,101,2020-08-30,Falcon 9,1600.000000,SSO,CCSFS SLC 40,True RTLS,4,True,True,True,5e9e3032383ecb267a34e7c7,5.0,5,B1059,-80.577366,28.561857,1,green
85,102,2020-09-03,Falcon 9,15600.000000,VLEO,KSC LC 39A,True ASDS,2,True,True,True,5e9e3032383ecb6bb234e7ca,5.0,12,B1060,-80.603956,28.608058,1,green
86,103,2020-10-06,Falcon 9,15600.000000,VLEO,KSC LC 39A,True ASDS,3,True,True,True,5e9e3032383ecb6bb234e7ca,5.0,13,B1058,-80.603956,28.608058,1,green
87,104,2020-10-18,Falcon 9,15600.000000,VLEO,KSC LC 39A,True ASDS,6,True,True,True,5e9e3032383ecb6bb234e7ca,5.0,12,B1051,-80.603956,28.608058,1,green
88,105,2020-10-24,Falcon 9,15600.000000,VLEO,CCSFS SLC 40,True ASDS,3,True,True,True,5e9e3033383ecbb9e534e7cc,5.0,12,B1060,-80.577366,28.561857,1,green
89,106,2020-11-05,Falcon 9,3681.000000,MEO,CCSFS SLC 40,True ASDS,1,True,False,True,5e9e3032383ecb6bb234e7ca,5.0,8,B1062,-80.577366,28.561857,1,green


In [75]:
# If zoomed in enough, the map now contains red and green markers for successful/unsuccessful landings

# Initialize a map centered at NASA Johnson Space Center
m = folium.Map(location=nasa_coordinate, zoom_start=10)

# Initialize the MarkerCluster
marker_cluster = MarkerCluster().add_to(m)

# Iterate through each row in the DataFrame and add a marker to the cluster
for idx, row in df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"FlightNumber: {row['FlightNumber']}, LaunchSite: {row['LaunchSite']}, Outcome: {'Success' if row['landing_class'] == 1 else 'Failure'}",
        icon=folium.Icon(color=row['marker_color'])
    ).add_to(marker_cluster)

# Display the map
m

In [78]:
# Save this dataframe to build machine learning models and make predictions on it
df.to_csv('f9_data_3.csv', index=False)